In [1]:
import random
from typing import TypedDict
import statistics
import numpy as np

In [2]:
PET_LEVEL_TO_DROPRATE_MULTIPLIER = {
    0:1.0,
    1:1.0,
    2:1.0,
    3:1.0,
    4:1.33,
    5:1.67,
    6:2.0,
    7:2.33,
    8:2.67,
    9:3.0,
    10:3.33,
    11:3.67,
    12:4.0,
}

class PetDropRatesDict(TypedDict):
    item: str
    dropRate: int

In [3]:
dragon_pet_drops: PetDropRatesDict = {
    "Hoard":100,
    "Random Card":150,
    "Event Card":500
}

ada_dwarf_drops: PetDropRatesDict = {
    "Adamantium":200,
}


In [4]:
def run_simulation(sim_days: int, pet_drops_rates:PetDropRatesDict, pet_level:int):
    simulation_steps = sim_days*24
    pet_level_drop_rate_modifier = PET_LEVEL_TO_DROPRATE_MULTIPLIER[pet_level]
    
    for key in pet_drops_rates.keys():
        chance_to_drop = pet_level_drop_rate_modifier/pet_drops_rates.get(key)
        #print(f'Chance to drop {key} is {pet_level_drop_rate_modifier}/{pet_drops_rates.get(key)} = {pet_level_drop_rate_modifier/pet_drops_rates.get(key)} ')

    drop_results = dict() # stores list of total days since last drop per each item
    steps_since_last_drop = dict() # stores temporary days since last drop count for each item 
    for drop_item in pet_drops_rates.keys():
        drop_results[drop_item] = list()
        steps_since_last_drop[drop_item] = 0

    last_drop_step = 0
    for sim_step in range(simulation_steps):
        if((sim_step - last_drop_step) < 24): # one day delay per drop
            #print(f'Sim Step is  {sim_step}, last item drop is {last_drop_step} continuing!')
            for drop_item in pet_drops_rates.keys(): #increase steps since last drop for each drop item
                steps_since_last_drop[drop_item] += 1
            
            continue #move to next sim step

        for drop_item in pet_drops_rates.keys():
            chance_to_drop = pet_level_drop_rate_modifier/pet_drops_rates.get(drop_item)
            item_dropped = random.random() < chance_to_drop
            if(item_dropped):
                #print(f'Item {drop_item} dropped after {steps_since_last_drop[drop_item]} steps')
                drop_results[drop_item].append(steps_since_last_drop[drop_item]+1) # add one more step to account for this step
                steps_since_last_drop[drop_item] = 0 # reset steps since the last time this item dropped
                last_drop_step = sim_step # set the global last time any item dropped
            else: # no item drop, add step to its count since last drop
                steps_since_last_drop[drop_item] += 1
    
    return drop_results
            

In [5]:
def run_n_sim(num_sims:int, years_per_sim:int, pet_drops:PetDropRatesDict, pet_level:int):
    all_results = dict()
    #combine drop time results from all simulations
    for n in range(num_sims):
        for drop_item in pet_drops.keys():
            all_results[drop_item] = list()
        
        drop_results = run_simulation(years_per_sim*365, pet_drops, pet_level)
        for drop_item in pet_drops.keys():
            all_results[drop_item].extend(drop_results.get(drop_item))
    return all_results

In [8]:
def print_n_sims_result(pet_drops:PetDropRatesDict, pet_level, all_results):
    # simulation output is in steps, convert steps to days while printing stats 
    for drop_item in pet_drops.keys():
        avg_drop_days = np.mean(all_results.get(drop_item))/24
        stddev_drop_days = np.std(all_results.get(drop_item))/24
        min_drop_days = np.min(all_results.get(drop_item))/24
        max_drop_days = np.max(all_results.get(drop_item))/24
        percentile95_drop_days = np.percentile(all_results.get(drop_item),95)/24

        drop_string = f'Stats [days] for {drop_item} at static level {pet_level}:\n ' + \
                    f'Avg: {round(avg_drop_days,2)} StdDev: {round(stddev_drop_days,2)} ' + \
                    f'Min: {round(min_drop_days,2)} Max: {round(max_drop_days,2)} ' + \
                    f'95%ile: {round(percentile95_drop_days,2)}'
        print(drop_string)

In [10]:
NUM_SIMS = 1000
SIM_YEARS = 10
SIM_PET_DROPS = dragon_pet_drops

for level in range(1,13):
    level_results = run_n_sim(NUM_SIMS, SIM_YEARS, SIM_PET_DROPS, level)
    print_n_sims_result(SIM_PET_DROPS, level, level_results)
    print()

Stats [days] for Hoard at static level 1:
 Avg: 5.78 StdDev: 5.26 Min: 1.0 Max: 35.96 95%ile: 17.75
Stats [days] for Random Card at static level 1:
 Avg: 9.85 StdDev: 8.68 Min: 1.0 Max: 70.83 95%ile: 26.02
Stats [days] for Event Card at static level 1:
 Avg: 26.59 StdDev: 28.41 Min: 1.12 Max: 201.83 95%ile: 80.83

Stats [days] for Hoard at static level 2:
 Avg: 6.1 StdDev: 5.48 Min: 1.0 Max: 40.21 95%ile: 17.4
Stats [days] for Random Card at static level 2:
 Avg: 8.84 StdDev: 8.02 Min: 1.0 Max: 76.88 95%ile: 23.49
Stats [days] for Event Card at static level 2:
 Avg: 25.02 StdDev: 24.49 Min: 1.0 Max: 157.58 95%ile: 70.53

Stats [days] for Hoard at static level 3:
 Avg: 6.03 StdDev: 4.72 Min: 1.0 Max: 26.88 95%ile: 15.57
Stats [days] for Random Card at static level 3:
 Avg: 8.57 StdDev: 7.97 Min: 1.0 Max: 82.83 95%ile: 23.98
Stats [days] for Event Card at static level 3:
 Avg: 30.05 StdDev: 28.99 Min: 1.38 Max: 193.75 95%ile: 80.83

Stats [days] for Hoard at static level 4:
 Avg: 4.79 St